# Load dependencies

In [1]:
from google.cloud import bigquery
import datetime
import pandas as pd

In [2]:
client = bigquery.Client()

In [3]:
limits = 'LIMIT 15000'
sql_events = f"""
WITH geovisits_h3_indexes AS (
    SELECT *, `carto-os-eu`.h3.LONGLAT_ASH3(longitude, latitude, 10) h3_idx
    FROM `ggo-ppos-bqgis.singlespot.geovisits_paris_matview`
)
select sptId, uuid, latitude, longitude, accuracy, eventId,  
arrival, departure, score, rank, category, feature, placeName, h3_idx 
FROM geovisits_h3_indexes {limits}
"""

In [4]:
rows = client.query(sql_events).result()
df = rows.to_dataframe()
df.head()

,sptId,uuid,latitude,longitude,accuracy,eventId,arrival,departure,score,rank,category,feature,placeName,h3_idx
0,7d21b764-daae-4eb2-8330-a1fa6d0367af,9cf02c39-87ce-4865-9a6d-a46adb199ed3,48.824074,2.373960,48.0,84ea804206129a24f1075da79ffea90da8cc6796e42be0...,2021-07-14 10:34:34 +0200,2021-07-14 10:39:35 +0200,0.346496,1,amenity,veterinary,Obsèques Animales,8a1fb4646327fff
1,8a8f87c1-7dd0-4376-babc-455dd7c3b8b0,7cab399a-d75c-48b1-9c5c-1cb44e4ae337,48.840264,2.341110,42.0,5a5786eed2f59a4119137d0ac227be2f599b7a08ce0224...,2021-07-13 07:41:42 +0200,2021-07-13 09:44:50 +0200,0.512278,1,office,association,Communauté Chrétienne Universitaire,8a1fb4671ccffff
2,c6aa8b4f-857a-42e6-9093-3a6743eaf127,6b8927ca-e0c8-4979-b242-b4e12946a53d,48.880470,2.327291,57.0,938ca94e31e6c22fd016224e2fc845ceab87442c65a1ee...,2021-07-09 11:10:18 +0200,2021-07-09 11:32:37 +0200,0.821422,1,shop,model,Au Pullman,8a1fb4666c37fff
3,1d32bf1b-8694-4598-a569-773c6659303f,ac975d15-555a-4131-8f8d-f90268474e8f,48.844595,2.376132,48.0,f-5f82826cad9aa807265dc87f08a4ab9112faba8b,2021-07-08 17:46:16 +0200,2021-07-08 18:15:50 +0200,0.365783,1,tourism,artwork,None,8a1fb464410ffff
4,b8fbec5b-0de2-4383-9276-de5f22c6fe8d,0b518296-1a3c-4d18-bef8-8f15ed056fe8,48.884469,2.344850,48.0,5a2fa95268e830ecbdbe990a242f467a5c586b1c6c27d0...,2021-07-10 16:02:04 +0200,2021-07-10 17:10:15 +0200,0.565314,1,shop,fabric,Les Coupons de Saint-Pierre,8a1fb4666baffff


In [5]:
df['sptId'] = df['sptId'].astype(str)
df['category'] = df['category'].astype(str)
df['feature'] = df['feature'].astype(str)
df['arrival'] = pd.to_datetime(df['arrival'], format='%Y-%m-%d %H:%M:%S', utc=True)
df['depature'] = pd.to_datetime(df['departure'], format='%Y-%m-%d %H:%M:%S', utc=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   sptId      15000 non-null  object             
 1   uuid       15000 non-null  object             
 2   latitude   15000 non-null  float64            
 3   longitude  15000 non-null  float64            
 4   accuracy   15000 non-null  float64            
 5   eventId    15000 non-null  object             
 6   arrival    15000 non-null  datetime64[ns, UTC]
 7   departure  15000 non-null  object             
 8   score      15000 non-null  float64            
 9   rank       15000 non-null  int64              
 10  category   15000 non-null  object             
 11  feature    15000 non-null  object             
 12  placeName  13433 non-null  object             
 13  h3_idx     15000 non-null  object             
 14  depature   15000 non-null  datetime64[ns, UTC]
dtypes:

In [6]:
df_w = pd.DataFrame(df[['h3_idx', 'category', 'feature']])
df_w['count'] = 1
df_w.head()


,h3_idx,category,feature,count
0,8a1fb4646327fff,amenity,veterinary,1
1,8a1fb4671ccffff,office,association,1
2,8a1fb4666c37fff,shop,model,1
3,8a1fb464410ffff,tourism,artwork,1
4,8a1fb4666baffff,shop,fabric,1


In [11]:
df_w['category'].unique()

array(['amenity', 'office', 'shop', 'tourism', 'leisure', 'building',
       'user', 'railway', 'historic', 'craft', 'natural', 'aeroway',
       'club', 'man_made', 'highway', 'landuse'], dtype=object)

In [25]:
filtered_categories = ['amenity', 'shop']
amenities_shops = df_w.loc[df_w['category'].isin(filtered_categories)]
amenities_shops.head()
amenities_shops[amenities_shops['category'] =='amenity']['feature'].unique()

array(['veterinary', 'coworking_space', 'motorcycle_parking', 'townhall',
       'driving_school', 'courthouse', 'events_venue', 'community_centre',
       'fuel', 'wlan', 'marketplace', 'cinema', 'fire_station', 'shelter',
       'public_building', 'bus_station', 'car_sharing', 'training',
       'conference_centre', 'arts_centre', 'construction', 'nightclub',
       'food_court', 'bureau_de_change', 'car_rental',
       'exhibition_centre', 'car_wash', 'money_transfer', 'photo_lab',
       'dojo', 'bicycle_repair_station', 'studio', 'music_school',
       'couture', 'events_center', 'student_accomodation',
       'administration', 'language_school', 'teahouse',
       'personal_service', 'bar', 'pub', 'bank', 'cafe', 'college',
       'library', 'theatre', 'pharmacy', 'fast_food', 'restaurant',
       'university', 'post_office', 'bicycle_rental'], dtype=object)

In [12]:
grouped_columns = ['category', 'feature']

In [22]:
h3_pivot = pd.DataFrame(pd.pivot_table(df_w,  columns=grouped_columns, index='h3_idx', values="count", aggfunc='sum', fill_value=0, margins = True, margins_name='total')).reset_index()


SyntaxError: keyword argument repeated: columns (887109103.py, line 1)

In [21]:
h3_pivot.head()

category           h3_idx aeroway                 amenity                   \
feature                   helipad heliport administration arts_centre bank   
0         8a1fb460d957fff       0        0              0           0    0   
1         8a1fb460d95ffff       0        0              0           0    0   
2         8a1fb462824ffff       0        0              0           0    0   
3         8a1fb462825ffff       0        0              0           0    0   
4         8a1fb462866ffff       0        0              0           0    0   

category                                                             ...  \
feature  bar bicycle_rental bicycle_repair_station bureau_de_change  ...   
0          0              0                      0                0  ...   
1          0              0                      0                0  ...   
2          0              0                      0                0  ...   
3          0              0                      0                0  ...   
4          0              0                      0                0  ...   

category tourism                                                        user  \
feature    hotel information museum theme_park viewpoint water_park zoo home   
0              0           0      0          0         0          0   0    4   
1              0           0      0          0         0          0   0    0   
2              0           0      0          0         0          0   0    0   
3              0           0      0          0         0          0   0    0   
4              0           0      0          0         0          0   0    0   

category      total  
feature  work        
0           0     6  
1           0     1  
2           0     1  
3           0     1  
4           0     1  

[5 rows x 319 columns]